In [18]:
import tensorflow as tf
import numpy as np
import json
import os
import zapisywacz

def zapisz_json(co,gdzie):
    f=open(gdzie,'w')
    f.write(json.dumps(co))

def wczytaj_json(skad):
    f=open(skad,'r')
    return json.loads(f.read())

def wyrwij_label(slownik):
    l= slownik.pop('l')
    return slownik, l

def rob_feature_columns(nazwa_zrodla):
    kategoryczne_typy = zapisywacz.wczytaj_typy_kategorycznych(nazwa_zrodla)
    numeryczne_typy = zapisywacz.wczytaj_typy_numerycznych(nazwa_zrodla)
    dlugosc_numerycznych = numeryczne_typy[len(numeryczne_typy) - 1][2]
    wyn = []
    wyn.append(tf.feature_column.numeric_column(
        key = 'n', shape=(dlugosc_numerycznych,)  ))
    for kat in kategoryczne_typy:
        nowy = tf.feature_column.categorical_column_with_vocabulary_list(
            key = kat[0],
            vocabulary_list = kat[1],
            dtype = tf.int32
        )
        hot = tf.feature_column.indicator_column(nowy)
        wyn.append(hot)
    return wyn

#nazwa_folderu to jak sie bedzie nazywac folder modelu
#glowne_zrodlo_danych to plik z ktorego beda pochodzic dane
# do policzenia danych do normalizacji danych
def tworz_folder(nazwa_folderu, glowne_zrodlo_danych, hidden_units, na_ilu_statystyki = 10000):
    assert not os.path.isdir(nazwa_folderu)
    os.mkdir(nazwa_folderu)
    def licz_statystyki(nazwa_zrodla, na_ilu):
        dataset = zapisywacz.parsuj_i_batchuj(nazwa_zrodla, na_ilu, 1)
        iterator = dataset.make_one_shot_iterator()
        tab = iterator.get_next()['n']
        mean, variance = tf.nn.moments(tab, axes=[0])
        return mean, variance
    
    mean, variance = licz_statystyki(glowne_zrodlo_danych, na_ilu_statystyki)
    print('policzylo mean oraz variance')
    with tf.Session() as sess:
        mean, variance = sess.run((mean, variance))
        np.savetxt(nazwa_folderu + '/mean', mean)
        np.savetxt(nazwa_folderu + '/var',variance)
    print('zapisalo je')
    feature_columns = rob_feature_columns(glowne_zrodlo_danych)
    model = tf.estimator.DNNClassifier(
        hidden_units = hidden_units,
        feature_columns = feature_columns,
        model_dir = nazwa_folderu + '/model',
        n_classes = 2
    )
    return model
    

def odczytaj_mean_var(nazwa_folderu):
    mean = np.loadtxt(nazwa_folderu + '/mean')
    var = np.loadtxt(nazwa_folderu + '/var')
    return mean, var

#odczytujemy juz istniejacy model
def wczytaj_model(nazwa_folderu):
    assert os.path.isdir(nazwa_folderu + '/model')
    return tf.estimator.DNNClassifier(model_dir = nazwa_folderu + '/model')

def input_fn(zrodlo_danych, folder_modelu, batch_size,  
         czy_shuffle, ile_threadow, czy_repeat, buffer_size,
         czy_cache ):
    mean, var = odczytaj_mean_var(folder_modelu)
    dataset = zapisywacz.parsuj_i_batchuj(zrodlo_danych, batch_size, ile_threadow)
    kategoryczne = zapisywacz.wczytaj_typy_kategorycznych(zrodlo_danych)
    if(czy_repeat):
        dataset = dataset.repeat()
    def normalizacja(slownik):
        slownik['n'] = (slownik['n'] - mean) / (var**0.5)
        return slownik
    def odczep_kategoryczne(slownik):
        kat_tab = slownik.pop('c')
        for i in range(len(kategoryczne)):
            slownik[kategoryczne[i][0]] = kat_tab[:, i]
        return slownik
    dataset = dataset.map(odczep_kategoryczne, num_parallel_calls = ile_threadow)
    dataset = dataset.map(normalizacja,num_parallel_calls = ile_threadow)
    dataset = dataset.map(wyrwij_label,num_parallel_calls = ile_threadow)
    if czy_shuffle:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.prefetch(1)
    if czy_cache:
        dataset = dataset.cache()
    return dataset






    


In [15]:
model = tworz_folder('model_probny', 'pierwszy',
             [30], na_ilu_statystyki = 10000)

policzylo mean oraz variance
zapisalo je
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_probny/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f14a71a2a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
dataset = input_fn('pierwszy','model_probny', 10,  
         True, 1, True, 10,
         True )

In [21]:
costam = dataset.make_one_shot_iterator()
sess = tf.InteractiveSession()
sess.run(costam.get_next())

/home/pawel/anaconda3/envs/root_tensorflow/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


({'n': array([[-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ],
         [-0.04469895, -0.04469948, -0.04469915, -0.04469931, -0.0446989 ]],
        dtype=float32), 'c': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])},
 array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]))